<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-Full-Stack LLM-101/05_3_OracleDB.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Retrieval Augmented Generation (RAG) Baseline
## Overview  
In this time, we will check the baseline code.
The goal of this project is to provide students with hands-on experience in handling and enhancing Large Language Models (LLMs) provided by [**Upstage**](https://www.upstage.ai) (Solar).

You can use any engineering method for improving benchmark performance excluding direct training (Fine-tuning).

*Collecting data directly related to the test set is considered cheating e.g., using MMLU-pro dataset or EWHA.pdf for KB*

# Baseline

In [23]:
#!pip3 install -qU python-dotenv PyPDF2 langchain langchain-community langchain-core langchain-text-splitters langchain_upstage oracledb python-dotenv

In [24]:
# Additional Contents : https://wikidocs.net/253106

In [25]:
# set parameters

file = open("info/api.txt", "r")
api_key = file.read()
file.close()

file = open("info/datapath.txt", "r")
data_path = file.read()
file.close()

file = open("info/resultspath.txt", "r")
results_path = file.read()
file.close()

In [26]:
from langchain_upstage import UpstageEmbeddings

# 쿼리 전용 임베딩 모델
query_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-query")

# 문장 전용 임베딩 모델
passage_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-passage")

In [27]:
# funcion to extract an answer from response

import re

def extract_answer(response):
    """
    extracts the answer from the response using a regular expression.
    expected format: "[ANSWER]: (A) convolutional networks"

    if there are any answers formatted like the format, it returns None.
    """
    pattern = r"\[ANSWER\]:\s*\((A|B|C|D|E)\)"  # Regular expression to capture the answer letter and text
    match = re.search(pattern, response)

    if match:
        return match.group(1) # Extract the letter inside parentheses (e.g., A)
    else:
        return extract_again(response)

def extract_again(response):
    pattern = r"\b[A-J]\b(?!.*\b[A-J]\b)"
    match = re.search(pattern, response)
    if match:
        return match.group(0)
    else:
        return None

## 1. build DB

In [28]:
from langchain_upstage import UpstageLayoutAnalysisLoader
import os
import numpy as np


UPSTAGE_API_KEY = api_key

# .npy 파일 로드 (타입==넘파이)
ewhaDB = np.load(data_path+f'embedding/full_ewha500.npy')

ewhaDB_embed = np.load(data_path+f'embedding/full_ewha500_embed.npy')
ewhaDB_embed = ewhaDB_embed.tolist()

## 2. Text Split

## 3. query-context matching

In [29]:
# read samples.csv file
import pandas as pd

def read_data(data_path):
    data = pd.read_csv(data_path)
    prompts = data['prompts']
    answers = data['answers']
    # returns two lists: prompts and answers
    return prompts, answers

In [30]:
prompts, answers = read_data(os.path.join(data_path, 'test_own_ewha.csv'))

In [31]:
testdata = pd.read_csv(data_path+'test_own_ewha.csv')

In [32]:
nowtest = pd.DataFrame(columns=['index', 'embed_ques', 'question', 'prompts', 'answers', 'top1', 'top2', 'top3', 'predict', 'ground' ])

for index, row in testdata.iterrows():
    #if index % 100 != 0 : continue # 일단 실험할 땐 100개 단위로 끊어서 가져옴
    q = row.prompts
    a = row.answers
    question = q.partition('(A)')[0]
    question = question.partition(') ')[2]
    q = q.partition(') ')[2]
    try : 
        embedded_query = passage_embeddings.embed_query(question) # 질문만 받아와서 embedding 하기
        nowtest.loc[len(nowtest)] = {'index':index, 'embed_ques' : embedded_query, 'question' : question, 'prompts' : q, 'answers' : a, 'top1': None, 'top2': None, 'top3': None, 'predict': None, 'gound': None }

    except :
        print(f'pass: {index}')
        continue 


In [33]:
nowtest

,index,embed_ques,question,prompts,answers,top1,top2,top3,predict,ground
0,0,"[-0.005329132080078125, 0.0017337799072265625,...",영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,(D),None,None,None,None,NaN
1,1,"[-0.00460052490234375, -0.0174713134765625, -0...",각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n,각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n(A) 이학사\n(B...,(E),None,None,None,None,NaN
2,2,"[0.007404327392578125, -0.0216522216796875, 0....",부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n,부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n(A) 현...,(B),None,None,None,None,NaN
3,3,"[-0.002162933349609375, -0.039215087890625, -0...",학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n,학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n(A) 시험 직후...,(B),None,None,None,None,NaN
4,4,"[0.0089111328125, -0.0259552001953125, -0.0064...",학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,(C),None,None,None,None,NaN
5,5,"[0.0016117095947265625, -0.019683837890625, -0...",학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n,학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n(A) 서...,(D),None,None,None,None,NaN
6,6,"[-0.018157958984375, -0.0157318115234375, -0.0...",학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?\n,학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?...,(C),None,None,None,None,NaN
7,7,"[0.0029163360595703125, -0.018707275390625, -0...",학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,(D),None,None,None,None,NaN
8,8,"[-5.4895877838134766e-05, -0.01477813720703125...",학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌...,학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌...,(C),None,None,None,None,NaN
9,9,"[-0.002269744873046875, 0.0017795562744140625,...",학칙 제50조(졸업)에서 복수전공 이수자가 요구되는 추가 조건은 무엇입니까?\n,학칙 제50조(졸업)에서 복수전공 이수자가 요구되는 추가 조건은 무엇입니까?\n(A...,(B),None,None,None,None,NaN


## 4. Prompt engineering

In [34]:
try : del [[wrongdf]]
except : pass
wrongdf = pd.DataFrame(columns=['questionNum', 'answer', '1st_pred', '2nd_pred'])

In [35]:
############# first PREDICTION ##########

from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage

 
llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    You are an assistant with expertise in Ewha University policies and history. Use the provided context to answer accurately.
    Let's think step by step.

    Context: {context}

    Question: {question}

    Final Answer:
        
    '''

)
chain1 = prompt_template | llm


In [36]:
import numpy as np

responses = []
for idx, row in nowtest.iterrows() : # 질문 받아오기 

    embed_ques= row.embed_ques

    # 유사도 기준 내림차순 정렬
    sorted_idx = (np.array(embed_ques) @ np.array(ewhaDB_embed).T).argsort()[::-1]

    nowtest.loc[idx, 'top1'] = ewhaDB[sorted_idx[0]]
    nowtest.loc[idx, 'top2'] = ewhaDB[sorted_idx[1]]
    nowtest.loc[idx, 'top3'] = ewhaDB[sorted_idx[2]]

    response = chain1.invoke({"question": row.prompts, "context": ewhaDB[sorted_idx[0]]}) # top1에 대해서는 바로 답변 출력
    responses.append(response.content)

    nowtest.loc[idx, 'predict'] = response.content

responses



['영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학점에 대한 정보는 제공된 문맥에서 명시적으로 언급되지 않았습니다. 따라서 주어진 선택지 (A) 인정 안됨, (B) 1학점, (C) 2학점, (D) 3학점 중 어느 것도 최종 답변으로 선택할 수 없습니다.',
 '(E) 인공지능학사',
 '(B) 구 학칙에 의하여 재학생이 졸업하는 연도까지 존속한다',
 'B',
 '학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 (D) 학술의 응용방법 연구입니다.',
 '(D) 강남구',
 '(C) 학생총회 요청',
 '(A) 건축학과는 추가 설계 실습을 포함하므로 더 긴 연한이 필요하다.',
 '(D) 해외 봉사로 인한 휴학: 3년',
 "(B) 전공별 종합시험 합격\n\nStep-by-step justification:\n\n1. The provided context refers to Ewha University's policies and history, particularly focusing on the classification of courses, types of credit hours, and other academic requirements.\n2. The question asks for the additional condition required by a double major student as mentioned in Article 50 (Graduation) of the school regulations.\n3. To answer this question accurately, I need to find the relevant information within the provided context.\n4. I found in the context that Article 35-2 (Special Cases for Credit Acquisition) mentions that specific subjec

In [37]:
nowtest.top1[1]

'당하게 할 수 있다.② 입학사정관의 조직과 운영에 관한 사항은 총장이 따로 정한다. (개정 2012.12.31.)\n[본조신설 2009.6.16.]제18조(입학허가) ① 입학(편입학 및 재입학 포함)은 해당 대학장의 제청으로 총장이 허가한\n다. 다만, 제15조제1항의 단서의 경우나 특별전형, 기타 입학에 관하여 긴급한 필요가 있는이화여자대학교 학칙경우에는 대학장의 제청을 거치지 아니할 수 있다. (개정 2015. 2.6.)\n② 총장은 입학자격을 갖추지 못하였거나 제1항의 입학사정에 사용된 전형자료에 허위나\n부정이 있다고 판단되는 경우에는 교무회의의 의결을 거쳐 합격취소 및 입학허가취소를 할\n수 있다. (신설 2005.7.6)\n③ 제2항의 합격취소 및 입학허가취소의 사유와 절차에 관한 사항은 총장이 따로 정한다.\n(신설 2005.7.6.)\n[제목개정 2015.2.6.]제19조(입학금 등) ① 입학이 허가된 자는 정해진 기일 내에 입학금 기타 납입금을 납입하여\n야 한다. (개정 1997.'

In [38]:
######### 정답 1차 확인 + wrong 뽑아내기 ######

# print accuracy

cnt = 0
wrong = []
for idx, (answer, response) in enumerate(zip(answers, nowtest['predict'])):
    print("-"*10)
    generated_answer = extract_answer(response)
    print(response)
    # check
    if generated_answer:
        print(f"idx: {idx} | generated answer: {generated_answer}, answer: {answer}")
    else:
        print("extraction fail")

    if generated_answer == None:
        wrong.append(idx+1)
        wrongdf.loc[len(wrongdf)] = {"questionNum":idx+1, "answer" : answer, "1st_pred":generated_answer, "2nd_pred":"-"}
        continue
    
    if generated_answer in answer:
        cnt += 1
    else : 
        wrong.append(idx+1)
        wrongdf.loc[len(wrongdf)] = {"questionNum":idx+1, "answer" : answer, "1st_pred":generated_answer, "2nd_pred":None}
        
acc = cnt/len(answers)*100
print(f"acc: {acc}%")
print()
print("1st wrong:", wrong)
wrongdf.loc[len(wrongdf)] = {"questionNum":"-", "answer" : "-", "1st_pred":acc, "2nd_pred":None}

----------
영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학점에 대한 정보는 제공된 문맥에서 명시적으로 언급되지 않았습니다. 따라서 주어진 선택지 (A) 인정 안됨, (B) 1학점, (C) 2학점, (D) 3학점 중 어느 것도 최종 답변으로 선택할 수 없습니다.
idx: 0 | generated answer: D, answer: (D)
----------
(E) 인공지능학사
idx: 1 | generated answer: E, answer: (E)
----------
(B) 구 학칙에 의하여 재학생이 졸업하는 연도까지 존속한다
idx: 2 | generated answer: B, answer: (B)
----------
B
idx: 3 | generated answer: B, answer: (B)
----------
학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 (D) 학술의 응용방법 연구입니다.
idx: 4 | generated answer: D, answer: (C)
----------
(D) 강남구
idx: 5 | generated answer: D, answer:  (D)
----------
(C) 학생총회 요청
idx: 6 | generated answer: C, answer:  (C)
----------
(A) 건축학과는 추가 설계 실습을 포함하므로 더 긴 연한이 필요하다.
idx: 7 | generated answer: A, answer:  (D)
----------
(D) 해외 봉사로 인한 휴학: 3년
idx: 8 | generated answer: D, answer:  (C)
----------
(B) 전공별 종합시험 합격

Step-by-step justification:

1. The provided context refers to Ewha University's policies and history, particularly focusing on the classifi

In [39]:
wrongdf

,questionNum,answer,1st_pred,2nd_pred
0,5,(C),D,None
1,8,(D),A,None
2,9,(C),D,None
3,11,(B),D,None
4,19,(C),D,None
5,22,(D),A,None
6,24,(C),B,None
7,26,(B),C,None
8,30,(A),C,None
9,31,(B),E,None


In [40]:
############# Second PREDICTION ##########

from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    You are an assistant with expertise in Ewha University policies and history. Use the provided context to answer accurately.
    Let's think step by step.

    Context: {context1} {context2}

    Question: {question}

    Final Answer:

    '''

)
chain2 = prompt_template | llm


response2 = []
for idx, row in nowtest.iterrows() : ############### 일단 지금은 100개 단위로 띄엄띄엄 test 중
    if idx not in wrong : continue
        
    response = chain2.invoke({"question": row.prompts, "context1": row.top1, "context2": row.top2}) # 선지 전까지 받아오기
    response2.append(response.content)
    nowtest.loc[idx, 'predict'] = response.content

response2

['(D) 강남구',
 '(E) 중대한 질병으로 인한 휴학: 총장의 판단에 따라 연장 가능',
 '(B) 전공별 종합시험 합격',
 '(C) 독립된 학과로 운영된다.',
 '(B) 20학점\n\n해설: 학칙 제45조(취득학점)에 따르면, 직전 학기 평균성적 3.75 이상을 받은 학생은 18학점 이상을 취득할 수 있으며, 20학점까지 취득할 수 있습니다. 따라서, 정답은 (B) 20학점입니다.',
 '(B) 제1전공의 졸업요건 학점을 충족해야 한다.',
 '(B) 시험 부정행위',
 '(E) 등록금이 과오납된 경우에는 금액을 반환할 수 없다.',
 '(E) 차이 없음',
 '(C) 학칙 개정은 총장의 단독 결재로 진행된다.',
 '(A) ㄱ,ㄹ',
 'D',
 '(B) 고등학교 졸업 성적은 입학전형자료로 사용될 수 없다.\n\n단계별 추론:\n\n1. 문맥은 이화여자대학교 학칙에 대한 내용을 포함하고 있으며, 특히 입학전형에 대한 제17조와 관련된 내용을 다룹니다.\n2. 제17조(입학전형)에서는 고등학교 졸업 정도 등을 기준으로 한 입학전형자료에 대해 언급하고 있습니다.\n3. 고등학교 졸업 성적도 입학전형자료로 사용될 수 있다고 언급되어 있으므로, (B) 번 선택지는 문맥과 일치하지 않습니다.\n4. 따라서, 제17조(입학전형)에 관한 설명으로 옳지 않은 것은 (B) 번입니다.',
 "(B) 첫 번째 과목의 성적은 '미기록', 두 번째 과목의 성적은 'U', 세 번째 과목의 성적은 'F'로 기록됩니다.",
 '(A) 충족했다 (129학점 이상)']

In [41]:
######### 정답 2차 확인 + wrong 뽑아내기 ######

# print accuracy

cnt = 0
wrong2 = []
for idx, (answer, response) in enumerate(zip(answers, nowtest['predict'])):
    print("-"*10)
    generated_answer = extract_answer(response)
    print(response)
    # check
    if generated_answer:
        print(f"idx: {idx} | generated answer: {generated_answer}, answer: {answer}")
    else:
        print("extraction fail")

    if generated_answer == None:
        wrong2.append(idx+1)
        wrongdf.loc[wrongdf['questionNum']==idx+1, '2nd_pred'] = "-"
        continue
    
    if generated_answer in answer:
        cnt += 1
    else : 
        wrong2.append(idx+1)
        wrongdf.loc[wrongdf['questionNum']==idx+1, '2nd_pred'] = generated_answer
        

acc = cnt/len(answers)*100
print(f"acc: {acc}%")
print()
print("2nd wrong:", wrong2)
wrongdf.loc[len(wrongdf)-1, '2nd_pred'] = acc

----------
영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학점에 대한 정보는 제공된 문맥에서 명시적으로 언급되지 않았습니다. 따라서 주어진 선택지 (A) 인정 안됨, (B) 1학점, (C) 2학점, (D) 3학점 중 어느 것도 최종 답변으로 선택할 수 없습니다.
idx: 0 | generated answer: D, answer: (D)
----------
(E) 인공지능학사
idx: 1 | generated answer: E, answer: (E)
----------
(B) 구 학칙에 의하여 재학생이 졸업하는 연도까지 존속한다
idx: 2 | generated answer: B, answer: (B)
----------
B
idx: 3 | generated answer: B, answer: (B)
----------
학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 (D) 학술의 응용방법 연구입니다.
idx: 4 | generated answer: D, answer: (C)
----------
(D) 강남구
idx: 5 | generated answer: D, answer:  (D)
----------
(C) 학생총회 요청
idx: 6 | generated answer: C, answer:  (C)
----------
(A) 건축학과는 추가 설계 실습을 포함하므로 더 긴 연한이 필요하다.
idx: 7 | generated answer: A, answer:  (D)
----------
(E) 중대한 질병으로 인한 휴학: 총장의 판단에 따라 연장 가능
idx: 8 | generated answer: E, answer:  (C)
----------
(B) 전공별 종합시험 합격
idx: 9 | generated answer: B, answer:  (B)
----------
(D) 교과목 담당교수의 허가 없이 수강 취소

성적 F는 휴학 중이거나, 수업 시간의 6분의 1 이상 결석, 중간시험 및 기말

In [42]:
wrongdf.set_index(keys=['questionNum'])
wrongdf

,questionNum,answer,1st_pred,2nd_pred
0,5,(C),D,D
1,8,(D),A,A
2,9,(C),D,E
3,11,(B),D,D
4,19,(C),D,D
5,22,(D),A,A
6,24,(C),B,B
7,26,(B),C,C
8,30,(A),C,C
9,31,(B),E,E


## groundedness check

### 질문할 때마다 답이 달라져서 안 할 거임 

In [43]:
import os
from langchain_upstage import UpstageGroundednessCheck # langchain_upstage==0.1.3
 
groundedness_check = UpstageGroundednessCheck()
 
request_input = {
    "context": f"Mauna Kea is an inactive volcano on the island of Hawai'i. Its peak is 4,207.3 m above sea level, making it the highest point in Hawaii and second-highest peak of an island on Earth.",
    "answer": "Mauna Kea is 5,207.3 meters tall.",
}
response = groundedness_check.invoke(request_input)
print(response)

ValueError: UPSTAGE_API_KEY must be set or passed